In [108]:
from orion.sources import S3Source
from orion.sources.io import read_csv, write_csv
from orion.sources import RedshiftSource
import pandas as pd
import numpy as np
from scipy import sparse

In [109]:
#source = RedshiftSource(query='SELECT * FROM publish.inventory_lookup WHERE productid IS NULL')
#df_stock = source.read_csv()

with S3Source(key="masters/uploads/customers/1560425511130_Peak_customers.csv", bucket="kilimanjaro-prod-datalake") as s3:
    df_cust = read_csv(s3)

df_trans = read_csv(S3Source(key="masters/uploads/transactions/1560426066385_Peak_transactions.csv", bucket="kilimanjaro-prod-datalake"))
df_prod = read_csv(S3Source(key="masters/uploads/product/1560425499995_Peak_product.csv", bucket="kilimanjaro-prod-datalake"))


/home/declanbarrycbc1/thesis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [110]:
# 30pc of transactions have keys that are not in df_cust
pd.Series(df_trans.customerkey.unique()).isin(df_cust.customerkey).sum()/len(df_trans.customerkey.unique())
# 45pc of item numbers not in df_prod
pd.Series(df_trans.itemnumber.unique()).isin(df_prod.itemnumber).sum()/len(df_trans.itemnumber.unique())

0.5487752814984791

In [111]:
df_cust.drop(df_cust[df_cust.gender=='unknown'].index, axis=0, inplace=True)

In [112]:
# Fix item number
df_prod.itemnumber = pd.to_numeric(df_prod.itemnumber, errors='coerce') 

In [113]:
# Remove keys not in customer and product tables
df_trans = df_trans.iloc[df_trans.customerkey[df_trans.customerkey.isin(df_cust.customerkey)].index,:]
df_trans.reset_index(inplace=True, drop=True)
df_trans = df_trans.iloc[df_trans.itemnumber[df_trans.itemnumber.isin(df_prod.itemnumber)].index,:]

# Reset index before changing keys to integers
df_trans.reset_index(inplace=True, drop=True)
df_prod.reset_index(inplace=True, drop=True)


In [114]:
# Drop nas
df_prod.dropna(inplace=True)
df_prod.itemnumber.unique().shape

(71993,)

In [115]:
print(df_cust.customerkey.unique().shape)
df_cust.dropna(inplace=True)
print(df_cust.customerkey.unique().shape)

print(df_trans.customerkey.unique().shape)
df_trans.dropna(inplace=True)
print(df_trans.customerkey.unique().shape)

(2425998,)
(1692766,)
(1605552,)
(1605201,)


In [116]:
# Drop redundant cols. Fabric all unknown, item colour less info than colourvalue
df_prod.drop(["itemstylecode", "itemquarter", "itemcolour", "fabric"], axis=1, inplace=True)

In [117]:
df_prod.reset_index(inplace=True, drop=True)

In [118]:
df_prod.iloc[:,1:].duplicated().sum()

45306

In [119]:
dups = df_prod.loc[df_prod.iloc[:,1:].duplicated(),:]

In [120]:
df_prod.drop(dups.index, inplace=True)

In [121]:
# Drop columns
df_trans.drop(["salestransactionkey", "salesordernumber","discountpercent", "grosssales",
              'orderdate', 'ordertime', 'shippingdate','grossprofit'], axis=1, inplace=True)

In [122]:
df = df_trans.merge(df_cust, on='customerkey', how = "outer")
df = df.merge(df_prod, on = "itemnumber")

In [123]:
# Rename and drop columns, nas
df.rename(columns={"customerkey":"user_id","itemnumber":"item_id","shipcountry":"country", "brandcode":"brand_id",
                  "colourvalue":"colour", }, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [124]:
df_trans.rename(columns={"customerkey":"user_id","itemnumber":"item_id"}, inplace=True)
df_cust.rename(columns={"customerkey":"user_id"}, inplace=True)
df_prod.rename(columns={"itemnumber":"item_id"}, inplace=True)

In [125]:
df.drop_duplicates(inplace=True)

In [126]:
df.shape,df.user_id.unique().shape, df.item_id.unique().shape

((952042, 16), (558421,), (12324,))

In [127]:
check = df[['user_id', 'item_id']].apply(lambda x:x.factorize()[0])

In [128]:
np.unique(np.array(check),axis=0).shape

(881203, 2)

### Take subset of data


In [129]:
cats = df.select_dtypes('object').columns
keys = df[cats].apply(lambda x: x.factorize()[1])
df[cats] = df[cats].apply(lambda x: x.factorize()[0])

In [130]:
df_sub = df.sample(frac = 0.10)

In [131]:
df_sub.shape

(95204, 16)

In [135]:
df1= df

In [132]:
[df[i].unique().shape for i in df.columns]

[(558421,),
 (62,),
 (11949,),
 (12324,),
 (2,),
 (10,),
 (2,),
 (2,),
 (328,),
 (8,),
 (58,),
 (3,),
 (13,),
 (7,),
 (3,),
 (78,)]

In [133]:
[df_sub[i].unique().shape for i in df_sub.columns]

[(86309,),
 (58,),
 (5537,),
 (10333,),
 (2,),
 (9,),
 (2,),
 (2,),
 (309,),
 (8,),
 (54,),
 (3,),
 (13,),
 (6,),
 (3,),
 (78,)]

In [104]:
df = df_sub

In [134]:
df.drop(['unitssold', 'pricetype'], axis=1, inplace=True)

In [152]:
len(df.item_id.unique())

12324

In [143]:
user_num = df.user_id.unique().shape[0]
item_num = df.item_id.unique().shape[0]

In [153]:
user_num, item_num

(558421, 12324)

In [155]:
#Renumber item ids
ints = [i for i in range(item_num)]
item_keys = [i for i in df.item_id.unique()]
item_key_dict = {i:j for i,j in zip( item_keys, ints)}

# Change keys to integers in dataframes
df.item_id = [item_key_dict[df.item_id.iloc[i]] for i in range(len(df))]

In [156]:
user_cols = ['user_id', 'country','postcode','loyaltyaccount','gender']
users = df[user_cols]
items = df[['item_id',"brand_id","category","colour", "divisioncode","itemcategorycode","itemfamilycode","itemseason","productgroup"]]

In [ ]:
interactions = df[['user_id', 'item_id']]

In [ ]:
users.drop_duplicates(inplace=True)
items.drop_duplicates(inplace=True)

In [ ]:
users.to_csv("users")
items.to_csv("items")
interactions.to_csv("interactions")

In [ ]:
# Train-test split
train = df.sample(frac = 0.8)
ind = df.index.isin(train.index)
test = df[~ind]

In [ ]:
train.head()

In [53]:
np.save("new_fa_train_data",train.to_numpy())
np.save("new_fa_test_data",test.to_numpy())

## Matrices

In [157]:
USER_NUM = len(df.user_id.unique())
country_num = len(df.country.unique())
postcode_num = len(df.postcode.unique())	
item_id_num = len(df.item_id.unique())
loyaltyaccount_num = len(df.loyaltyaccount.unique())
gender_num = len(df.gender.unique())
brand_id_num = len(df.brand_id.unique())
category_num = len(df.category.unique())
colour_num = len(df.colour.unique())
divisioncode_num = len(df.divisioncode.unique())
itemcategorycode_num = len(df.itemcategorycode.unique())
itemfamilycode_num = len(df.itemfamilycode.unique())
itemseason_num = len(df.itemseason.unique())
productgroup_num = len(df.productgroup.unique())

In [158]:
print(USER_NUM,
country_num,
postcode_num,
item_id_num,
loyaltyaccount_num,
gender_num,
brand_id_num,
category_num,
colour_num,
divisioncode_num,
itemcategorycode_num,
itemfamilycode_num,
itemseason_num,
productgroup_num)

558421 62 11949 12324 2 2 328 8 58 3 13 7 3 78


### Item Attribute matrix

In [159]:
features = [   brand_id_num, category_num, colour_num, divisioncode_num, 
            itemcategorycode_num, itemfamilycode_num, itemseason_num, productgroup_num]
names = [  'brand_id',
         'category', 'colour', 'divisioncode', 'itemcategorycode',
         'itemfamilycode', 'itemseason', 'productgroup']
dic = dict(zip(names,features))

In [162]:
matrices = []
for i in dic.keys():
    mat = np.zeros((item_num, dic[i]), dtype = np.int)    
    pair = df.loc[:,["item_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[1 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [163]:
[(matrices[i]).shape for i in range(len(matrices))]

[(12324, 328),
 (12324, 8),
 (12324, 58),
 (12324, 3),
 (12324, 13),
 (12324, 7),
 (12324, 3),
 (12324, 78)]

In [164]:
ia_matrix = np.concatenate((matrices), axis=1)

In [165]:
np.save('new_fa_ia_matrix.npy', ia_matrix)

In [167]:
matrices[0].shape

(12324, 328)

### User- attribute matrix

In [168]:
matrices = []
for i in dic.keys():
    mat = np.zeros((USER_NUM, dic[i]), dtype = np.int)
    pair = df.loc[:,["user_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [169]:
fa_matrix = np.concatenate((matrices), axis=1)

In [170]:
fa_matrix.shape[1]

498

In [171]:
[(matrices[i]).shape for i in range(len(matrices))]

[(558421, 328),
 (558421, 8),
 (558421, 58),
 (558421, 3),
 (558421, 13),
 (558421, 7),
 (558421, 3),
 (558421, 78)]

In [172]:
print(np.unique(ia_matrix, axis=0).shape, np.unique(fa_matrix, axis=0).shape)

(12324, 498) (152501, 498)


In [173]:
fa_matrix.shape

(558421, 498)

In [174]:
np.save('new_fa_ua_matrix', fa_matrix)

### User - attribute Matrices

In [144]:
features = [   country_num, postcode_num, gender_num, loyaltyaccount_num]
names = [ "country", "postcode", "gender", "loyaltyaccount" ]
dic = dict(zip(names,features))

In [70]:
matrices = []
for i in dic.keys():
    mat = np.zeros((item_num, dic[i]), dtype = np.int)
    pair = df.loc[:,["item_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 ... 0 0 0]
[1 1]
[0 1]


In [71]:
[(matrices[i]).shape for i in range(len(matrices))]

[(10393, 57), (10393, 5504), (10393, 2), (10393, 2)]

In [72]:
user_atts_matrix = np.concatenate((matrices), axis=1)

In [73]:
user_atts_matrix.shape

(10393, 5565)

In [74]:
np.save("item_attributes", user_atts_matrix)

In [75]:
# Ignore postcode

In [76]:
features = [   country_num, gender_num, loyaltyaccount_num]
names = [ "country", "gender", "loyaltyaccount" ]
dic = dict(zip(names,features))

In [77]:
matrices = []
for i in dic.keys():
    mat = np.zeros((USER_NUM, dic[i]), dtype = np.int)
    pair = df.loc[:,["user_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1]
[0 1]


In [78]:
[(matrices[i]).shape for i in range(len(matrices))]

[(86364, 57), (86364, 2), (86364, 2)]

In [79]:
user_atts_matrix = np.concatenate((matrices), axis=1)

In [80]:
user_atts_matrix.shape

(86364, 61)

In [81]:
np.save("user_attributes_npc", user_atts_matrix)

### User - Item Matrix

In [82]:
mat = np.zeros((user_num, item_num), dtype = np.int32)
pair = df.loc[:,["user_id", "item_id"]]
M = np.unique(pair, axis=0)
for m in M:
    mat[m[0], m[1]] = 1
print(mat[1])

[0 1 0 ... 0 0 0]


In [83]:
np.save("fa_ui_matrix", mat)